In [1]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 15
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

from matplotlib.colors import LinearSegmentedColormap

from scipy.stats import binom

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


In [2]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

In [3]:
latestsubdir = list(os.walk(path_data))[0][1][-1]
latestdir = path_data + latestsubdir

dfPCR = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
dfPCR = dfPCR.iloc[:-2]
dfPCR['NewPositive'] = pd.to_numeric(dfPCR['NewPositive'].astype(str).apply(lambda x: x.replace('.','')))
dfPCR['Tested'] = pd.to_numeric(dfPCR['Tested'].astype(str).apply(lambda x: x.replace('.','')))
dfPCR['PosPct'] = pd.to_numeric(dfPCR['PosPct'].astype(str).apply(lambda x: x.replace(',','.')))
dfPCR['Date'] =  pd.to_datetime(dfPCR.Date,format='%Y-%m-%d')
testDates = dfPCR['Date']


dfAG = pd.read_csv(latestdir+'/Test_pos_over_time_antigen.csv',delimiter = ';',dtype=str)
dfAG = dfAG.iloc[:-2]
dfAG['NewPositive'] = pd.to_numeric(dfAG['NewPositive'].astype(str).apply(lambda x: x.replace('.','')))
dfAG['Tested'] = pd.to_numeric(dfAG['Tested'].astype(str).apply(lambda x: x.replace('.','')))
dfAG['PosPct'] = pd.to_numeric(dfAG['PosPct'].astype(str).apply(lambda x: x.replace(',','.')))
dfAG['Date'] =  pd.to_datetime(dfAG.Date,format='%Y-%m-%d')


dfConf = pd.read_csv(latestdir+'/Antigentests_pr_dag.csv',delimiter = ';',dtype=str)
dfConf['Andel_AGpos_PCRkonf_pct'] = pd.to_numeric(dfConf['Andel_AGpos_PCRkonf_pct'].astype(str).apply(lambda x: x.replace(',','.')))
dfConf['Andel_AGminusPCRkonf_pct'] = pd.to_numeric(dfConf['Andel_AGminusPCRkonf_pct'].astype(str).apply(lambda x: x.replace(',','.')))
dfConf['Dato'] =  pd.to_datetime(dfConf.Dato,format='%Y-%m-%d')
dfConf['AG_testede'] =  dfConf.AG_testede.astype(int)
dfConf['AG_pos'] =  dfConf.AG_pos.astype(int)
# dfConf.tail()

In [4]:
display(dfPCR.tail(5))
display(dfAG.tail(5))
display(dfConf.tail(5))

,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
713,2022-01-09,16403,133.602,12.3,14.310,147912,55.076.193
714,2022-01-10,23404,187.949,12.5,20.067,208016,55.284.209
715,2022-01-11,22771,175.011,13.0,18.396,193407,55.477.616
716,2022-01-12,22586,166.638,13.6,18.007,184645,55.662.261
717,2022-01-13,11138,80.609,13.8,8.376,88985,55.751.246


,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
641,2022-01-09,5216,195.670,2.7,6.210,201880,53.364.553
642,2022-01-10,6221,232.936,2.7,7.700,240636,53.605.189
643,2022-01-11,5900,229.309,2.6,7.386,236695,53.841.884
644,2022-01-12,5800,222.979,2.6,7.290,230269,54.072.153
645,2022-01-13,6526,244.160,2.7,7.855,252015,54.324.168


,Dato,AG_testede,AG_pos,AGpos_PCRkonf,Andel_AGpos_PCRkonf_pct,AGpos_minusPCRkonf,Andel_AGminusPCRkonf_pct,AGpos_PCRpos,AGposPCRneg,AGnegPCRpos,AGnegPCRneg
339,2022-01-06,245152,5059,4054,80.134414,1005,19.865586,3505,423,3813,29940
340,2022-01-07,238193,4635,3692,79.654800,943,20.345200,3231,341,3820,26917
341,2022-01-08,143015,3891,3125,80.313544,766,19.686456,2819,223,3395,19546
342,2022-01-09,201973,5781,4809,83.186300,972,16.813700,4356,327,5669,30114
343,2022-01-10,240809,6813,5688,83.487450,1125,16.512550,5139,411,6659,32817


In [8]:
fig,allAxes = plt.subplots(3,2,sharex=True,tight_layout=True)

meanWidth = 7

ax = allAxes[0,0]
ax.plot(dfPCR.Date,dfPCR.NewPositive,'k.:',linewidth=0.5,markersize=2)
ax.plot(rnTime(dfPCR.Date,meanWidth),rnMean(dfPCR.NewPositive,meanWidth),'k')
ax.set_ylim(bottom=0)
ax.set_title('PCR')
ax.set_ylabel('Antal positive')

ax = allAxes[0,1]
ax.plot(dfAG.Date,dfAG.NewPositive,'k.:',linewidth=0.5,markersize=2)
ax.plot(rnTime(dfAG.Date,meanWidth),rnMean(dfAG.NewPositive,meanWidth),'k')
ax.set_ylim(bottom=0)
ax.set_title('AG')

ax = allAxes[1,0]
ax.plot(dfPCR.Date,dfPCR.Tested,'k.:',linewidth=0.5,markersize=2)
ax.plot(rnTime(dfPCR.Date,meanWidth),rnMean(dfPCR.Tested,meanWidth),'k')
ax.set_ylim(bottom=0)
ax.set_ylabel('Antal testede')

ax = allAxes[1,1]
ax.plot(dfAG.Date,dfAG.Tested,'k.:',linewidth=0.5,markersize=2)
ax.plot(rnTime(dfAG.Date,meanWidth),rnMean(dfAG.Tested,meanWidth),'k')
ax.set_ylim(bottom=0)

ax = allAxes[2,0]
ax.plot(dfPCR.Date,dfPCR.PosPct,'k.:',linewidth=0.5,markersize=2)
ax.plot(rnTime(dfPCR.Date,meanWidth),rnMean(dfPCR.PosPct,meanWidth),'k')
ax.set_ylim(bottom=0)
ax.set_ylabel('Positiv procent')
ax.set_ylim(top=20)

ax = allAxes[2,1]
ax.plot(dfAG.Date,dfAG.PosPct,'k.:',linewidth=0.5,markersize=2)
ax.plot(rnTime(dfAG.Date,meanWidth),rnMean(dfAG.PosPct,meanWidth),'k')
ax.set_ylim(bottom=0)
ax.set_ylim(top=5)

ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))


ax.set_xlim(left=np.datetime64('2021-06-01'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(18779.0, 19040.85)

In [6]:
dfConf.tail()

,Dato,AG_testede,AG_pos,AGpos_PCRkonf,Andel_AGpos_PCRkonf_pct,AGpos_minusPCRkonf,Andel_AGminusPCRkonf_pct,AGpos_PCRpos,AGposPCRneg,AGnegPCRpos,AGnegPCRneg
339,2022-01-06,245152,5059,4054,80.134414,1005,19.865586,3505,423,3813,29940
340,2022-01-07,238193,4635,3692,79.654800,943,20.345200,3231,341,3820,26917
341,2022-01-08,143015,3891,3125,80.313544,766,19.686456,2819,223,3395,19546
342,2022-01-09,201973,5781,4809,83.186300,972,16.813700,4356,327,5669,30114
343,2022-01-10,240809,6813,5688,83.487450,1125,16.512550,5139,411,6659,32817


In [7]:
fig,ax1 = plt.subplots()

ax1.plot(dfConf.Dato,dfConf.AG_pos,'k.:',markersize=2,linewidth=1)
ax1.plot(rnTime(dfConf.Dato,meanWidth),rnMean(dfConf.AG_pos,meanWidth),'k')
ax1.plot(dfAG.Date,dfAG.NewPositive,'m.:',markersize=2,linewidth=1)
ax1.plot(rnTime(dfAG.Date,meanWidth),rnMean(dfAG.NewPositive,meanWidth),'m')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …